In [1]:
%matplotlib inline

import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn; seaborn.set()
from sklearn.decomposition import PCA


## Read input data

In [2]:
pd.set_option('display.max_columns', None)  
def getHour(timestr):
    return timestr.split(':')[0]
def getDate(datestr):
    return datestr.split('/')[1]
def correctTimestamp(timestamp):
    if type(timestamp) == 'str':
        return timestamp.replace('T', '')
    else:
        return timestamp

def cleanDF(dataframe):
    if 'accident_index' in dataframe.columns:
        dataframe['accident_index'] = dataframe['accident_index'].apply(correctTimestamp)
    if 'time' in dataframe.columns:
        dataframe['time'] = dataframe['time'].apply(getHour)
    if 'date' in dataframe.columns:
        dataframe['date'] = dataframe['date'].apply(getDate)

In [3]:
casualty_2020_df=pd.read_csv(
    "https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-casualty-2020.csv"
)
cleanDF(casualty_2020_df)
casualty_2020_df

,accident_index,accident_year,accident_reference,vehicle_reference,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile
0,2020010219808,2020,010219808,1,1,3,1,31,6,3,9,5,0,0,0,0,1,4
1,2020010220496,2020,010220496,1,1,3,2,2,1,3,1,1,0,0,0,0,1,2
2,2020010220496,2020,010220496,1,2,3,2,4,1,3,1,1,0,0,0,0,1,2
3,2020010228005,2020,010228005,1,1,3,1,23,5,3,5,9,0,0,0,0,1,3
4,2020010228006,2020,010228006,1,1,3,1,47,8,2,4,1,0,0,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115579,2020991027064,2020,991027064,2,1,1,1,11,3,2,0,0,0,0,0,1,1,2
115580,2020991029573,2020,991029573,1,1,3,2,63,9,3,10,1,0,0,0,0,1,10
115581,2020991030297,2020,991030297,2,1,1,1,38,7,2,0,0,0,0,0,5,2,9
115582,2020991030900,2020,991030900,2,1,1,1,76,11,3,0,0,0,0,0,1,1,9


In [7]:
vehicle_2020_df=pd.read_csv(
    "https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-vehicle-2020.csv"
)
cleanDF(vehicle_2020_df)
vehicle_2020_df

/Users/dsherwin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type
0,2020010219808,2020,10219808,1,9,9,5,1,5,0,0,0,0,0,0,4,9,6,2,32,6,1968,2,6,AUDI Q5,4,1
1,2020010220496,2020,10220496,1,9,0,4,2,6,0,2,0,0,0,0,1,1,2,1,45,7,1395,1,2,AUDI A1,7,1
2,2020010228005,2020,10228005,1,9,0,18,-1,-1,0,0,0,0,0,0,1,1,6,3,-1,-1,-1,-1,-1,-1,-1,-1
3,2020010228006,2020,10228006,1,8,0,18,1,5,0,0,0,0,0,0,1,1,1,1,44,7,1798,8,8,TOYOTA PRIUS,2,1
4,2020010228011,2020,10228011,1,9,0,18,3,7,9,1,0,0,0,0,1,1,6,1,20,4,2993,2,4,BMW 4 SERIES,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167370,2020991030297,2020,991030297,1,9,0,7,8,2,0,6,0,0,0,0,3,1,1,1,57,9,1968,2,2,AUDI A5,7,1
167371,2020991030297,2020,991030297,2,5,0,16,6,2,0,1,0,0,0,0,1,1,5,1,38,7,1301,1,2,KTM 1290 SUPERDUKE,9,2
167372,2020991030900,2020,991030900,1,9,0,7,8,2,0,6,0,0,0,0,3,1,6,2,68,10,1995,2,1,BMW X3,5,1
167373,2020991030900,2020,991030900,2,1,0,18,6,2,4,8,0,0,0,0,1,1,6,1,76,11,-1,-1,-1,-1,9,1


In [5]:
accident_2020_df=pd.read_csv(
    "https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-accident-2020.csv"
)
cleanDF(accident_2020_df)
accident_2020_df

/Users/dsherwin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2020010219808,2020,10219808,521389.0,175144.0,-0.254001,51.462262,1,3,1,1,02,3,09,10,E09000032,E09000032,6,0,6,20,0,-1,6,0,9,9,1,9,9,0,0,1,3,2,E01004576
1,2020010220496,2020,10220496,529337.0,176237.0,-0.139253,51.470327,1,3,1,2,04,2,13,9,E09000022,E09000022,3,3036,6,20,9,2,6,0,0,4,1,1,1,0,0,1,1,2,E01003034
2,2020010228005,2020,10228005,526432.0,182761.0,-0.178719,51.529614,1,3,1,1,01,4,01,1,E09000033,E09000033,5,0,6,30,3,1,6,0,0,0,4,1,2,0,0,1,1,2,E01004726
3,2020010228006,2020,10228006,538676.0,184371.0,-0.001683,51.541210,1,2,1,1,01,4,01,17,E09000025,E09000025,3,11,6,30,0,-1,6,0,0,4,4,1,1,0,0,1,1,2,E01003617
4,2020010228011,2020,10228011,529324.0,181286.0,-0.137592,51.515704,1,3,1,2,01,4,02,1,E09000033,E09000033,3,40,6,30,3,4,5,0,0,0,4,1,1,0,0,1,1,2,E01004763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91194,2020991027064,2020,991027064,343034.0,731654.0,-2.926320,56.473539,99,2,2,1,08,4,14,918,S12000042,S12000042,4,959,6,30,1,4,4,959,0,0,1,1,1,0,0,1,1,-1,-1
91195,2020991029573,2020,991029573,257963.0,658891.0,-4.267565,55.802353,99,3,1,1,11,6,15,922,S12000011,S12000011,6,0,9,30,3,4,6,0,0,0,1,1,1,0,0,1,2,-1,-1
91196,2020991030297,2020,991030297,383664.0,810646.0,-2.271903,57.186317,99,2,2,1,04,4,12,910,S12000033,S12000033,4,979,6,60,8,-1,6,0,0,0,1,1,1,0,0,2,1,-1,-1
91197,2020991030900,2020,991030900,277161.0,674852.0,-3.968753,55.950940,99,3,2,1,12,3,14,932,S12000044,S12000044,6,0,6,30,3,4,6,0,0,0,1,1,1,0,0,1,2,-1,-1


#### We have 91199 accidents but 167375 vechicles and 115584 casualties in the data
#### Need to determine the primary vechicle in the crash data

In [11]:
vehicle_2020_df[vehicle_2020_df['vehicle_reference'] ==1]

,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type
0,2020010219808,2020,10219808,1,9,9,5,1,5,0,0,0,0,0,0,4,9,6,2,32,6,1968,2,6,AUDI Q5,4,1
1,2020010220496,2020,10220496,1,9,0,4,2,6,0,2,0,0,0,0,1,1,2,1,45,7,1395,1,2,AUDI A1,7,1
2,2020010228005,2020,10228005,1,9,0,18,-1,-1,0,0,0,0,0,0,1,1,6,3,-1,-1,-1,-1,-1,-1,-1,-1
3,2020010228006,2020,10228006,1,8,0,18,1,5,0,0,0,0,0,0,1,1,1,1,44,7,1798,8,8,TOYOTA PRIUS,2,1
4,2020010228011,2020,10228011,1,9,0,18,3,7,9,1,0,0,0,0,1,1,6,1,20,4,2993,2,4,BMW 4 SERIES,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167367,2020991027064,2020,991027064,1,9,0,16,3,7,0,3,0,0,0,0,1,1,1,1,27,6,-1,-1,-1,-1,2,1
167369,2020991029573,2020,991029573,1,9,0,1,5,1,5,1,0,0,0,0,2,1,3,2,39,7,1598,1,7,NISSAN QASHQAI,10,1
167370,2020991030297,2020,991030297,1,9,0,7,8,2,0,6,0,0,0,0,3,1,1,1,57,9,1968,2,2,AUDI A5,7,1
167372,2020991030900,2020,991030900,1,9,0,7,8,2,0,6,0,0,0,0,3,1,6,2,68,10,1995,2,1,BMW X3,5,1


We have 91152 accidents where there is a primary vechicle
That leaves 47 accidents where there is not a primary vechicle

In [12]:
vehicle_2020_df[vehicle_2020_df['vehicle_reference'] !=1]

,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type
7,2020010228014,2020,10228014,2,9,3,18,1,5,0,1,0,0,0,0,1,1,6,3,19,4,2143,2,4,MERCEDES C CLASS,-1,-1
9,2020010228017,2020,10228017,2,9,0,3,3,7,0,6,0,0,0,0,0,1,6,1,26,6,2993,2,0,BMW X5,2,1
11,2020010228018,2020,10228018,2,9,0,2,0,0,0,2,0,0,0,0,2,1,6,3,-1,-1,1984,1,1,AUDI Q5,-1,-1
13,2020010228020,2020,10228020,2,9,0,4,7,3,0,1,0,0,0,0,1,1,6,1,40,7,1600,1,8,BMW 1 SERIES,5,1
16,2020010228024,2020,10228024,2,9,0,3,5,1,0,1,1,0,0,0,1,1,6,1,41,7,1986,2,14,AUDI A4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167363,2020991024039,2020,991024039,2,9,0,2,6,2,0,0,0,0,0,0,3,1,2,1,52,8,1968,2,3,SEAT LEON,4,3
167365,2020991024209,2020,991024209,2,9,0,13,4,8,0,8,0,0,7,0,1,1,5,1,36,7,1968,2,3,VOLKSWAGEN PASSAT,10,3
167368,2020991027064,2020,991027064,2,1,0,5,1,5,0,6,0,0,0,0,4,1,5,1,11,3,-1,-1,-1,-1,2,1
167371,2020991030297,2020,991030297,2,5,0,16,6,2,0,1,0,0,0,0,1,1,5,1,38,7,1301,1,2,KTM 1290 SUPERDUKE,9,2


In [ ]:
full_df = full_df.merge(casualty_2020_df, on='accident_index')


In [ ]:
full_df

In [ ]:
no_loc_data = full_df.drop(['accident_reference_x', 'location_easting_osgr', 'location_northing_osgr', 'longitude', 'latitude', 'local_authority_ons_district','local_authority_highway', 'lsoa_of_accident_location', 'accident_reference_y', 'accident_reference', 'generic_make_model' ], axis = 1)
no_loc_data 

In [ ]:
no_loc_data['time'] = no_loc_data['time'].apply(getHour)

In [ ]:
no_loc_data['date'] = no_loc_data['date'].apply(getMonth)

In [ ]:
no_loc_data  

In [ ]:
no_loc_data = no_loc_data.set_index('accident_index')

In [ ]:
no_loc_data = no_loc_data.drop([], axis=1)

In [ ]:
no_loc_data.head()

In [ ]:
clean_pca = PCA(n_components=50)
principalComponents = clean_pca.fit_transform(no_loc_data)
principalDf = pd.DataFrame(data = principalComponents
             )
principalDf

In [ ]:
pd.Series(clean_pca.explained_variance_ratio_).plot(logy=True)

In [ ]:
factors_2020 = accident_2020_df.merge(vehicle_2020_df, on='accident_index')

In [ ]:
factors_2020 = factors_2020.drop(['accident_reference_x', 'location_easting_osgr', 'location_northing_osgr', 'longitude', 'latitude', 'local_authority_ons_district','local_authority_highway', 'lsoa_of_accident_location', 'accident_reference_y',  'generic_make_model' ], axis = 1)

In [ ]:
factors_2020

In [ ]:
factors_2020['time'] = factors_2020['time'].apply(getHour)
factors_2020['date'] = factors_2020['date'].apply(getMonth)

In [ ]:
pd.set_option('display.max_columns', None)  
factors_2020.head()

In [ ]:
factors_2020[factors_2020['accident_index'].str.contains("T")==True]

In [ ]:
factors_2020.head()

In [ ]:
clean_pca = PCA(n_components=52)
principalComponents = clean_pca.fit_transform(factors_2020)
principalDf = pd.DataFrame(data = principalComponents
             )
principalDf

In [ ]:
pd.Series(clean_pca.explained_variance_ratio_).plot(logy=True)

In [ ]:
sns.heatmap(numerical_only_df.corr(), annot=True, cmap="coolwarm") 